In [ ]:
!pip install git+https://github.com/maciejkula/spotlight

  Cloning https://github.com/maciejkula/spotlight to /tmp/pip-req-build-631k51ks
  Running command git clone --filter=blob:none --quiet https://github.com/maciejkula/spotlight /tmp/pip-req-build-631k51ks
  Resolved https://github.com/maciejkula/spotlight to commit 75f4c8c55090771b52b88ef1a00f75bb39f9f2a9
  Preparing metadata (setup.py) ... done
  Created wheel for spotlight: filename=spotlight-0.1.6-py3-none-any.whl size=33898 sha256=b11c5bbe0c064f32fee6758935065148b8d47fd545b1a57ecd0913df9701013b
  Stored in directory: /tmp/pip-ephem-wheel-cache-y4aa62ju/wheels/80/b2/99/3a84fb1d343d3168ffb55710aa8c9fcde83c50720205fc3e95
Successfully built spotlight
  Attempting uninstall: spotlight
    Found existing installation: spotlight 3.3.0
    Uninstalling spotlight-3.3.0:
      Successfully uninstalled spotlight-3.3.0


In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import gc
import torch
from spotlight.factorization.explicit import ExplicitFactorizationModel
from spotlight.evaluation import precision_recall_score
from spotlight.interactions import Interactions
from spotlight.factorization.implicit import ImplicitFactorizationModel
from spotlight.losses import *

In [2]:
df = pd.read_csv('./vseros-rank-otb/train_df.tsv',sep='\t')
df.head(5)

,Unnamed: 0,community_id,description,customers_count,messages_count,type,region_id,themeid,business_category,business_parent,customer_id,status,join_request_date
0,0,00dbc35ec26c5b6452a3259194b2d2f74eae7141d2bc75...,"Ванга сказала: ""Выживет пчела - выживет челове...",2966,1,7,NaN,NaN,NaN,NaN,947224211267aefcc2e3e9c524fdf46ce329bc638e8bf1...,A,NaN
1,1,06107f1eae304c45d3e6324cc86f7d39662793a53b159b...,NaN,8982,2,7,1.042403e+10,246.0,NaN,NaN,b935c3390d82df612de19591d2dd16e1eff8e660746bb3...,A,NaN
2,2,06107f1eae304c45d3e6324cc86f7d39662793a53b159b...,NaN,8982,2,7,1.042403e+10,246.0,NaN,NaN,ff873dee9234ee290d92eb6c15d7179aabf0653dbed3b3...,A,NaN
3,3,06107f1eae304c45d3e6324cc86f7d39662793a53b159b...,NaN,8982,2,7,1.042403e+10,246.0,NaN,NaN,737ef0e849feda6cdfd350d19fcedaf17aebb757c0a1fc...,A,NaN
4,4,097e4dae68029ecc3c6183ed4386aff71dcac227cbaff7...,Читать внимательно!!! Причины попадания в чёрн...,1035,1,7,1.041779e+10,NaN,FAN_CLUB,BLOG,780fb8c178af972826ac0f84e42ac4c6c1dcdeeec0c4dd...,A,NaN


In [3]:
users_inv_mapping = dict(enumerate(df['customer_id'].unique()))
users_mapping = {v: k for k, v in users_inv_mapping.items()}

items_inv_mapping = dict(enumerate(df['community_id'].unique()))
items_mapping = {v: k for k, v in items_inv_mapping.items()}
len(users_mapping),len(items_mapping)

(1568446, 149114)

In [4]:
test_idx = np.load('./test_index.npy')
train_idx = list(set(df.index) - set(test_idx))

In [5]:
test_df = df.loc[test_idx]
train_df = df.loc[train_idx]

In [6]:
del df
gc.collect()

11

In [7]:
def get_dataset(df,user_col,item_col,weight_col,users_mapping,items_mapping):
    return Interactions(user_ids=df[user_col].map(users_mapping.get).values,
                        item_ids=df[item_col].map(items_mapping.get).values,
                        weights=df[weight_col].values,
                        ratings = np.ones(len(df))
                       )

In [8]:
train_ds = get_dataset(df=train_df,
                       user_col='customer_id',
                       item_col='community_id',
                       weight_col='customers_count',
                       users_mapping=users_mapping,
                       items_mapping=items_mapping)

In [9]:
test_ds = get_dataset(df=test_df,
                       user_col='customer_id',
                       item_col='community_id',
                       weight_col='customers_count',
                       users_mapping=users_mapping,
                       items_mapping=items_mapping)

In [42]:
params = {'n_iter':2,
          'embedding_dim':512,
          'loss':'logistic',
          'batch_size':2048*2,
          'l2':0.1,
          #'learning_rate':1e-3,
          'optimizer_func':torch.optim.AdamW,
          'use_cuda':True,
          'sparse':False,
          'random_state':np.random.RandomState(56)}

model = ExplicitFactorizationModel(**params)

In [43]:
model.fit(train_ds,verbose=True)

Epoch 0: loss 0.13424235857221006
Epoch 1: loss 0.004302098162825401


In [13]:
test_users = test_df['customer_id'].map(users_mapping.get).unique()
test_items = train_df['community_id'].map(items_mapping.get).unique()

In [22]:
ranks.shape

(148963,)

In [25]:
def get_preds_single(model,test_users,users_mapping,items_inv_mapping,k=10):
    preds = []
    for user in tqdm(test_users):
        user_id = users_mapping[user]
        ranks = model.predict(user_id)
        best_idx = np.argpartition(ranks, -1*k)[-1*k:]
        best_users = [items_inv_mapping.get(x) for x in best_idx]
        preds += [best_users]
    return preds

In [ ]:
preds = get_preds_single(model,
                  test_df['customer_id'],
                  users_mapping,
                  items_inv_mapping,
                  k=10)

  0%|          | 0/420000 [00:00<?, ?it/s]

In [29]:
def apk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [30]:
test_group = test_df[['community_id','customer_id']].groupby('customer_id').agg(lambda x:x)
test_group['community_id'] = test_group['community_id'].apply(lambda x:x.tolist())

In [31]:
act = test_group['community_id'].tolist()
mapk(act,preds,k=7)

0.00031878117913832204